# Import library

In [1]:
%load_ext autoreload
%autoreload 2

# ===============================
# 1. Import 핵심 라이브러리 (우선순위: numerical/scientific/data)
import numpy as np           # 수치 해석용 (필수)
import pandas as pd          # 데이터프레임 처리용
import math                  # 수학 함수
from tqdm import tqdm        # 진행 상황 시각화
import matplotlib.pyplot as plt           # 플롯
import matplotlib.ticker as ticker        # 플롯 축 설정
import CoolProp.CoolProp as CP            # 열역학/물성치 구할 때 필요
import plotly.graph_objects as go

# 2. 커스텀 라이브러리 (src 폴더 내)
import sys
sys.path.append('src')       # 커스텀 모듈 경로 추가

import dhw_ex_model as dem       # 모델 엔진 (src/dhw_ex_model)
import dartwork_mpl as dm        # 플롯 스타일 커스텀

# 3. 플롯 스타일 적용 (dartwork_mpl)
dm.use_style('dmpl_light')   # 다트워크 플롯 스타일 적용 (라이트)


Load colors...
Load colormaps...


# 0. 준비

In [2]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.0),
    'legend': dm.fs(-1.5),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-1.5),
    'cbar_label': dm.fs(-1.5),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-2.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)

# 1. Constant

In [3]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3

# 2. 시뮬레이션 설정

## 2.1 Water use schedule

In [4]:
entries1 = [
        # --- 0:00 - 6:00 (사용 0) ---
        # --- 아침 피크 ---
        ("6:00", "6:30", 0.5),
        ("6:30", "7:00", 0.9),
        ("7:00", "7:30", 1.0),  # <--- Peak (1.0)
        ("7:30", "8:00", 0.9),
        ("8:00", "8:30", 0.6),
        ("8:30", "9:00", 0.3),
        
        # --- 주간 (최소 0.05, 점심 피크) ---
        ("9:00", "11:30", 0.05), # 2.5시간 (5개 구간)
        ("11:30", "12:00", 0.2), # 점심 준비/손 씻기
        ("12:00", "12:30", 0.4), # 식사/설거지
        ("12:30", "13:00", 0.4), # 설거지/양치
        ("13:00", "13:30", 0.2), # 마무리
        ("13:30", "17:00", 0.05),# 3.5시간 (7개 구간)
        
        # --- 저녁 피크 ---
        ("17:00", "17:30", 0.3),
        ("17:30", "18:00", 0.5),
        ("18:00", "18:30", 0.8),
        ("18:30", "19:00", 0.9),
        ("19:00", "19:30", 0.8),
        ("19:30", "20:00", 0.7),
        
        # --- 저녁 (감소) ---
        ("20:00", "20:30", 0.5),
        ("20:30", "21:00", 0.4),
        ("21:00", "21:30", 0.3),
        ("21:30", "22:00", 0.2),
        # --- 22:00 - 24:00 (사용 0) ---
    ]

entries2 = [
    ("0:00", "0:30", 0.0),
    ("0:30", "1:00", 0.0),
    ("1:00", "1:30", 0.0),
    ("1:30", "2:00", 0.0),
    ("2:00", "2:30", 0.0),
    ("2:30", "3:00", 0.0),
]

## 2.3 GSHPB model define

In [5]:
# 2. 모델 객체 생성 (모든 설정값을 __init__에 전달)
gshpb_model = dem.GroundSourceHeatPumpBoiler(
    V_disp_cmp        = 0.00005,
    eta_cmp_isen      = 0.7,
    Ts                = 16.0,
    T0                = 0.0,       # 외기 온도 5°C로 설정
    T_tank_w_setpoint = 65.0,      # 설정 온도 60°C로 변경
    dV_w_serv_m3s     = 0.0001    # 최대 유량 변경
      # ... (나머지 파라미터는 기본값 사용)
)

## 2.3 Simulation time and scale

In [6]:
sim_period_sec = 1000 * dem.h2s
dt = 60
time = np.arange(0, sim_period_sec, dt)

# 3. Simulation 실행 및 csv 저장

In [9]:
# 4. 시뮬레이션 실행

sim_results_df = gshpb_model.run_simulation(
    simulation_period_sec = sim_period_sec, 
    dt_s = dt,                         
    T_tank_w_init_C = 60.0,
    result_save_csv_path = 'result/gshpb_simulation_results2.csv',
    schedule_entries = entries1,
    # save_ph_diagram = True,
    # snapshot_save_path='./video'
)

GSHPB Simulating:   0%|          | 0/60000 [00:00<?, ?it/s]

GSHPB Simulating:   1%|          | 311/60000 [00:00<00:57, 1037.09it/s]/home/habin/Codes/DHW_EX_model/src/dhw_ex_model/utility_functions.py:137: RuntimeWarning: divide by zero encountered in scalar divide
  lbs = 1 / np.sqrt(4*as_*t)
GSHPB Simulating:   2%|▏         | 1160/60000 [00:15<13:17, 73.81it/s] 


KeyboardInterrupt: 

# 4. Visualization

## 4.1 Data import

In [10]:
df = pd.read_csv('result/gshpb_simulation_results2.csv', index_col=0)

## 4.2 Data series to array

In [11]:
is_on = df['is_on'].to_numpy()

Q_ref_cond  = df['Q_ref_cond'].to_numpy()
Q_ref_evap  = df['Q_ref_evap'].to_numpy()
Q_LMTD_cond = df['Q_LMTD_cond'].to_numpy()
Q_LMTD_evap = df['Q_LMTD_evap'].to_numpy()

Q_b         = df['Q_b'].to_numpy()
Q_cond_load = df['Q_cond_load'].to_numpy()
E_cmp       = df['E_cmp'].to_numpy()
E_pmp       = df['E_pmp'].to_numpy()
m_dot_ref   = df['m_dot_ref'].to_numpy()
cmp_rpm     = df['cmp_rpm'].to_numpy()

Ts = df['Ts'].to_numpy(); Ts_K = dem.C2K(Ts);
T0 = df['T0'].to_numpy(); T0_K = dem.C2K(T0);
T_sup_w = df['T_sup_w'].to_numpy(); T_sup_w_K = dem.C2K(T_sup_w);
T_tank_w  = df['T_tank_w'].to_numpy(); T_tank_w_K = dem.C2K(T_tank_w);
T_serv_w  = df['T_serv_w'].to_numpy(); T_serv_w_K = dem.C2K(T_serv_w);
T_b       = df['T_b'].to_numpy(); T_b_K = dem.C2K(T_b);
T_b_f     = df['T_b_f'].to_numpy(); T_b_f_K = dem.C2K(T_b_f);
T_b_f_in  = df['T_b_f_in'].to_numpy(); T_b_f_in_K = dem.C2K(T_b_f_in);
T_b_f_out = df['T_b_f_out'].to_numpy(); T_b_f_out_K = dem.C2K(T_b_f_out);

dV_b_f        = df['dV_b_f'].to_numpy()          # m3/s
dV_w_serv     = df['dV_w_serv'].to_numpy()       # m3/s
dV_w_sup_tank = df['dV_w_sup_tank'].to_numpy()   # m3/s
dV_w_sup_mix  = df['dV_w_sup_mix'].to_numpy()    # m3/s

T1 = df['T1'].to_numpy(); T1_K = dem.C2K(T1); P1 = df['P1'].to_numpy(); h1 = df['h1'].to_numpy(); s1 = df['s1'].to_numpy(); x1 = df['x1'].to_numpy();  
T2 = df['T2'].to_numpy(); T2_K = dem.C2K(T2); P2 = df['P2'].to_numpy(); h2 = df['h2'].to_numpy(); s2 = df['s2'].to_numpy(); x2 = df['x2'].to_numpy(); 
T3 = df['T3'].to_numpy(); T3_K = dem.C2K(T3); P3 = df['P3'].to_numpy(); h3 = df['h3'].to_numpy(); s3 = df['s3'].to_numpy(); x3 = df['x3'].to_numpy(); 
T4 = df['T4'].to_numpy(); T4_K = dem.C2K(T4); P4 = df['P4'].to_numpy(); h4 = df['h4'].to_numpy(); s4 = df['s4'].to_numpy(); x4 = df['x4'].to_numpy(); 

## 4.3 Other exergy parameters

In [12]:
COP = Q_cond_load/(E_cmp+E_pmp)
Q_tank_l = gshpb_model.UA_tank * (T_tank_w_K - T0_K)
X_tank_l = (1 - T0_K/T_tank_w_K) * Q_tank_l

# --- 1. 엑서지 흐름/전달 변수 계산 (NumPy 배열) ---
# (기본 물질전달·에너지전달 변수 → 엑서지 변환)
X1 = x1 * m_dot_ref
X2 = x2 * m_dot_ref
X3 = x3 * m_dot_ref
X4 = x4 * m_dot_ref

X_pmp = E_pmp
X_cmp = E_cmp

# --- 2. 엑서지 생성/전달/손실 항 계산 ---
X_g   = (1 - T0_K/Ts_K) * Q_b              # 지중 원천 엑서지 유입량
X_b   = (1 - T0_K/T_b_K) * Q_b             # 지중(Borehole)에서의 엑서지량
X_b_f = (1 - T0_K/T_b_f_K) * Q_b           # 지중열 교환기 유출부 엑서지량

X_ref_evap = (1 - T0_K/T_b_f_K) * Q_ref_evap  # 증발기(냉매쪽) 엑서지 유입량
X_ref_cond = (1 - T0_K/T_tank_w_K) * Q_ref_cond # 응축기(냉매→저탕조) 엑서지 유입량

X_tank_sup_w = dem.calc_exergy_flow(G = c_w * rho_w * dV_w_sup_tank, T = T_sup_w_K, T0 = T0_K) # 탱크 공급수 엑서지
X_tank_w     = dem.calc_exergy_flow(G = c_w * rho_w * dV_w_sup_tank, T = T_tank_w_K, T0 = T0_K) # 탱크 온수부 엑서지
X_mix_sup_w  = dem.calc_exergy_flow(G = c_w * rho_w * dV_w_sup_mix,  T = T_sup_w_K,   T0 = T0_K) # 믹싱밸브 공급수 엑서지
X_mix_serv_w = dem.calc_exergy_flow(G = c_w * rho_w * dV_w_serv,     T = T_serv_w_K,   T0 = T0_K) # 믹싱밸브 서비스 엑서지

# --- 3. 산키 밸런스용 '손실' 및 네트워크 엑서지 계산 (전개 순서대로) ---
# (앞단→뒷단, 또는 각 블록 분기별)
Xc_g        = X_g - X_b                     # 지중 → 보어홀 손실
Xc_b        = X_b - X_b_f                   # 보어홀 → 지중열교환기 손실 
Xc_GHE      = (X_b_f + X_pmp) - X_ref_evap  # 유체 루프(exergy 유입+펌프입력 - 증발기 엑서지)
Xc_evap     = (X_ref_evap + X4) - X1        # 증발기 전후 손실
Xc_cmp      = (X1 + X_cmp) - X2             # 압축기 통과 후 손실
Xc_ref_cond = X2 - X3 - X_ref_cond          # 응축기에서의 손실
Xc_exp      = X3 - X4                       # 팽창밸브 손실

Xc_tank     = X_ref_cond + X_tank_sup_w - X_tank_w               # 탱크 전후 손실
Xc_mix      = (X_tank_w + X_mix_sup_w-X_mix_serv_w)                           # 믹싱부 전체 엑서지(총계)

# ------------------------------------------
# values (산키용 순서)에서 필요한 추가 변수 정의
# values = [
#   X_b, Xc_g, X_b_f, Xc_b, X_pmp, X_ref_evap, Xc_GHE,
#   X_flow_X3_to_X4, X_flow_X4_to_X1, Xc_evap, X_cmp, 
#   X_flow_X1_to_X2, Xc_cmp, X_flow_X2_to_X3, X_ref_cond, 
#   Xc_ref_cond, Xc_exp, 
#   X_tank_sup_w, X_flow_to_tank_w, Xc_tank, 
#   X_mix_sup_w, X_mix_serv_w, Xc_mix
# ]

# 순환흐름(유량 기반 exergy flow) 변수들 정의
X_flow_X1_to_X2 = X1 + X_cmp
X_flow_X2_to_X3 = X_flow_X1_to_X2 - X_ref_cond - Xc_ref_cond
X_flow_X3_to_X4 = X_flow_X2_to_X3 - Xc_exp
X_flow_X4_to_X1 = X_flow_X3_to_X4 + (X_b_f - Xc_GHE + X_pmp) - Xc_evap

/tmp/ipykernel_3985266/1769710613.py:1: RuntimeWarning: invalid value encountered in divide
  COP = Q_cond_load/(E_cmp+E_pmp)


## 4.4 Plot function

In [11]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정|
# =============================================================================
def plot_simple_graph(df_column, time, xlabel, ylabel, xmin = 0, xmax=24, Kelvin=False, color = 'dm.blue5', savepath = None):
    # 온도 데이터를 섭씨(°C)로 변환하여 플로팅
    if Kelvin:
        y = dem.K2C(df_column)
    else:
        y = df_column
    x = time * dem.s2h  # 초를 시간으로 변환

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    
    # --- 마이너 틱 설정 ---

    # X축: 주 틱 사이를 5개의 간격으로 나눔 (마이너 틱 4개 생성)
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    # Y축: 주 틱 사이를 2개의 간격으로 나눔 (마이너 틱 1개 생성)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    ax.plot(x, y, color=color, linewidth=0.5,)
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))

    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(np.nanmin(y)*0.9, np.nanmax(y)*1.1)
    # ax.set_ylim(50, 55)
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox = [0, 1, 0.02, 1])
    plt.savefig(f'{savepath}.png')
    dm.save_and_show(fig)
    plt.close()

def plot_multi_graph(df_columns, legends, time, xlabel, ylabel, linestyles=None, colors=None, xmin=0, xmax=24, Kelvin=False, savepath=None, scatter = False):
    """
    여러 개의 데이터 열을 받아 하나의 그래프에 플로팅하는 함수.
    scatter=True이면 점 그래프(scatter plot)를 그립니다.

    Args:
        *df_columns: 플로팅할 데이터 열들 (예: df['col1'], df['col2'], ...)
        legends (list or tuple): 각 데이터 열에 해당하는 레이블 리스트
        time (array-like): x축 시간 데이터
        ... (기존 인자들과 동일)
        colors (list or tuple, optional): 각 플롯에 사용할 색상 리스트. 기본값 None.
        scatter (bool): True이면 scatter plot, False이면 line plot을 그립니다.
    """
    if len(df_columns) != len(legends):
        raise ValueError("데이터 컬럼의 개수와 레이블의 개수가 일치해야 합니다.")

    # --- 1. 그래프 기본 설정 ---
    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    x = time * dem.s2h  # 초를 시간으로 변환
    
    # 기본 색상 리스트 (colors 인자가 주어지지 않을 경우 사용)
    if colors is None:
        colors = ['dm.blue5', 'dm.orange5', 'dm.green5', 'dm.red5', 'dm.violet5',
                  'dm.gray5', 'dm.yellow5', 'dm.cyan5']
    
    # --- 2. 모든 데이터를 바탕으로 Y축 min/max 계산 ---
    global_min = np.inf
    global_max = -np.inf
    
    processed_ys = []
    for col in df_columns:
        y = dem.K2C(col) if Kelvin else col
        processed_ys.append(y)
        if not y.empty:
            global_min = min(global_min, np.nanmin(y))
            global_max = max(global_max, np.nanmax(y))
            
    # --- 3. 반복문을 통해 여러 데이터 플로팅 (수정된 부분) ---
    for i, y_data in enumerate(processed_ys):
        
        current_color = colors[i % len(colors)]
        current_label = legends[i]

        if scatter:
            # scatter=True이면 ax.scatter() 호출
            ax.scatter(x, y_data, 
                       color=current_color,
                       label=current_label,
                       s=1.5, # 점 크기 (필요에 따라 조절)
                       alpha=0.3 # 투명도 (점이 겹칠 경우 유용)
                      )
        else:
            # scatter=False이면 기존 ax.plot() 호출
            current_linestyle = linestyles[i % len(linestyles)] if linestyles is not None else ['-', '--', '-.', ':'][i % 4]
            ax.plot(x, y_data, 
                    color=current_color,
                    linewidth=0.8, 
                    label=current_label,
                    linestyle=current_linestyle,
                   )

    # --- 4. 축 및 레이아웃 설정 ---
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    global_int = global_max - global_min
    
    # Y축 범위 설정 (무한대 값 방지)
    if np.isinf(global_min) or np.isinf(global_max):
        ax.set_ylim(0, 1) # 기본값 설정 또는 다른 적절한 처리
    else:
        ax.set_ylim(global_min - global_int * 0.3, global_max + global_int * 0.3) # 전체 데이터 기준 ylim 설정
    
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])

    ax.legend(ncol = 6, fontsize = fs['legend']) # 범례 표시
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    
    if savepath:
        plt.savefig(f'{savepath}.png')
        
    dm.save_and_show(fig)
    plt.close()

## 4.5 Quick plot

In [140]:
plot_multi_graph(
    df_columns = [
        df['T_b'],
        df['T_b_f'], 
        df['T_b_f_out'],
                  ],
    legends = [
        'Borehole surface [°C]',
        'Borehole surface [°C]',
        'Borehole heat transfer rate [W]'
               ],
    time = time,
    xlabel = 'Time (hours)',
    ylabel = 'Temperature [°C]',
    xmin = 0,
    xmax = 24,
    # savepath = 'results/gshpb_temperatures'
)

In [69]:
xmin_idx = int(100 * dem.h2s/dt)
xmax_idx = int(120 * dem.h2s/dt)
np.max(df['T_b_f_in'].iloc[xmin_idx:xmax_idx])

nan

# 5. Presentation Plot

## 5.1 Hot water tank temp and water schedule 

In [13]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.

X = time * dem.s2h  # 시간 [h]
Y1 = df['dV_w_serv'] * dem.m32L / dem.s2m  # 온수 사용량 [L/min]

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Hour of day [h]'
Y_LABEL = 'Hot water usage [L/min]'

xmin1, xmax1, xint1, xmar1 = 0, 24, 4, 0
ymin1, ymax1, yint1, ymar1 = 0, 8, 2, 0
color_ax1 = 'dm.red'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

ax1.plot(X, Y1, label='', linewidth= LW[2], color=color_ax1 + '4')

# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y_LABEL, fontsize=fs['label'], labelpad=pad['label'])


ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - 0.2, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 온수 사용 시점 시각적으로 표시
dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig('figure/HeatPump_model/water_use_schedule.svg', dpi=900, transparent=True)
plt.savefig('figure/HeatPump_model/water_use_schedule.png', dpi=900, transparent=True)
dm.save_and_show(fig)

## 5.2 Borehole wall temperature and Ground exergy consumption

In [13]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'result/gshpb_simulation_results2.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = time * dem.s2h  # 시간 [h]
Y1 = T_b
Y2 = Xc_g

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Borehole wall temperature [°C]'
Y2_LABEL = 'Ground exergy consum rate [W]'
xmin1 = 72
xmax1, xint1, xmar1 = xmin1+48, 12, 0
ymin1, ymax1, yint1, ymar1 = 12, 16, 1, 0
ymin1_tw, ymax1_tw, yint1_tw = 0, 70, 10
color_ax1 = 'dm.red'
color_ax1_twin = 'dm.green'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))


# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')


ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

# 온수 사용 시점 시각적으로 표시
ax1_twin = ax1.twinx()
# ax1_twin.plot(X, Y2, color=color_ax1_twin + '4', label='Hot water use [L/min]', linestyle='-', linewidth=LW[1])
ax1_twin.fill_between(X, 0, Y2, color=color_ax1_twin + '3', alpha=0.5, label='Hot water use [L/min]', zorder = 0, rasterized=True)
ax1.plot(X, Y1, label='', linewidth= LW[2], color=color_ax1 + '4', rasterized=True)
ax1_twin.set_ylabel(Y2_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')

ax1_twin.set_ylim(0, ymax1_tw)

ax1_twin.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))
for k in ['left', 'top', 'bottom']:
    ax1_twin.spines[k].set_visible(False)
ax1_twin.spines['right'].set_visible(True)
ax1_twin.spines['right'].set_color(color_ax1_twin + '6')
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)
ax1_twin.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))


ax1.set_zorder(2)             # 메인 축을 위로
ax1_twin.set_zorder(1)        # 트윈 축을 아래로
ax1.patch.set_alpha(0.0)      # 메인 축 배경을 투명하게 (아니면 ax1.set_facecolor('none'))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig(f'figure/HeatPump_model/Borehole_wall_temp_&_ground_Xc_{xmin1}.svg', dpi=900, transparent=True)
plt.savefig(f'figure/HeatPump_model/Borehole_wall_temp_&_ground_Xc_{xmin1}.png', dpi=900, transparent=True)
dm.save_and_show(fig)

## Fig. 1 COP and tank water temp

In [10]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'result/gshpb_simulation_results2.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = time*dem.s2h  # 시간 [h]
Y1 = T_tank_w
Y2 = COP

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Tank water temperature [°C]'
Y2_LABEL = 'COP [ - ]'

xmin1 = 934
xmax1, xint1, xmar1 = xmin1 + 48, 12, 0
ymin1, ymax1, yint1, ymar1 = 30, 70, 10, 0
ymin1_tw, ymax1_tw, yint1_tw = 2.0, 4.0, 0.5
color_ax1 = 'dm.pink'
color_ax1_twin = 'dm.indigo'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))


# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')


ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

# 온수 사용 시점 시각적으로 표시
ax1_twin = ax1.twinx()
# ax1_twin.plot(X, Y2, color=color_ax1_twin + '4', label='Hot water use [L/min]', linestyle='-', linewidth=LW[1])
mask = Y2 != 0
ax1_twin.scatter(X[mask], Y2[mask], color=color_ax1_twin + '3', label='COP [ - ]', s=0.5, zorder=3, rasterized=True)
# ax1_twin.fill_between(X, 0, Y2, color=color_ax1_twin + '2', alpha=0.5, label='Hot water use [L/min]', zorder = 0)
ax1.plot(X, Y1, label='', linewidth= LW[1], color=color_ax1 + '4', rasterized=True)
ax1_twin.set_ylabel(Y2_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')

ax1_twin.set_ylim(ymin1_tw, ymax1_tw)

ax1_twin.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))
for k in ['left', 'top', 'bottom']:
    ax1_twin.spines[k].set_visible(False)
ax1_twin.spines['right'].set_visible(True)
ax1_twin.spines['right'].set_color(color_ax1_twin + '6')
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)
ax1_twin.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.set_zorder(2)             # 메인 축을 위로
ax1_twin.set_zorder(1)        # 트윈 축을 아래로
ax1.patch.set_alpha(0.0)      # 메인 축 배경을 투명하게 (아니면 ax1.set_facecolor('none'))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_COP_{xmin1}.svg', dpi=900, transparent=True)
plt.savefig(f'figure/HeatPump_model/Tank_water_temp_&_COP_{xmin1}.png', dpi=900, transparent=True)
dm.save_and_show(fig)

## 5.2 Exergy and out

In [11]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'result/gshpb_simulation_results2.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = time * dem.s2h  # 시간 [h]
Y1 = df['Xs']
Y2 = df['Xb']
# Y3 = df['X_evap_ref']

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Exergy rate [W]'

xmin1 = 960
xmax1, xint1, xmar1 = xmin1+24, 6, 0
ymin1, ymax1, yint1, ymar1 = 200, 360, 40, 0
color1 = 'dm.pink'
color2 = 'dm.violet'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(8), dm.cm2in(4)))


# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

mask = Y2 != 0
ax1.scatter(X[mask], Y1[mask], color=color1 + '3', label='From ground', s=0.5, zorder=3, rasterized=True)
ax1.scatter(X[mask], Y2[mask], color=color2 + '3', label='To borehole wall', s=0.5, zorder=3, rasterized=True)
# ax1.scatter(X[mask], Y3[mask], color='dm.orange4', label='Exergy into evaporator', s=0.5, zorder=3, rasterized=True)

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))


ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', labelsize=fs['tick'], which='minor', pad=pad['tick'],)


# Legend를 위한 더미 플랏 생성. 더 진하고 큰 포인트를 사용한다.
scatter1 = ax1.scatter([], [], c=color1 + '7', s=1)
scatter2 = ax1.scatter([], [], c=color2 + '7', s=1)

_, labels = ax1.get_legend_handles_labels()
handles = [scatter1, scatter2]
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=2,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

dm.simple_layout(fig, bbox=[0, 1, 0.1, 1], margins=[0.0, 0.0, 0.0, 0.0])
plt.savefig(f'figure/HeatPump_model/Xs_&_Xb_{xmin1}.svg', dpi=900, transparent=True)
plt.savefig(f'figure/HeatPump_model/Xs_&_Xb_{xmin1}.png', dpi=900, transparent=True)
dm.save_and_show(fig)

KeyError: 'Xs'

In [36]:
# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'result/gshpb_simulation_results2.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = time * dem.s2h  # 시간 [h]
Y1 = T_tank_w
Y2 = T_b_f_in
Y3 = df['E_cmp'] * dem.W2kW
Y4 = np.array([200] * df['E_pmp'].shape[0]) * dem.W2kW # pmp

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Tank water temperature [°C]'
Y3_LABEL = 'Electricity input [kW]'

xmin1 = 0
xmax1 = xmin1 + 48 
xint1, xmar1 = 12, 0
ymin1, ymax1, yint1, ymar1 = 30, 70, 10, 0
ymin1_tw, ymax1_tw, yint1_tw = 0, 4, 1
color_ax1 = 'dm.pink'
color_ax1_twin = 'dm.green'
color_ax1_twin2 = 'dm.lime'
# ===============================================================

# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(10), dm.cm2in(5.5)))

# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')

ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

# --- 레전드 제거 ---
# (기존 legend, 더미 플랏 등 모두 제거)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

# 온수 사용 시점 시각적으로 표시
ax1_twin = ax1.twinx()
mask = Y3 != 0
ax1_twin.scatter(X[mask], Y3[mask], color=color_ax1_twin + '4', label='Compressor', s=0.5, zorder=3, rasterized=True)
ax1_twin.scatter(X[mask], Y4[mask], color=color_ax1_twin2 + '4', label='Pump', s=0.5, zorder=3, rasterized=True)
ax1.plot(X, Y1, label='', linewidth= LW[1], color=color_ax1 + '4', linestyle='-', rasterized=True,)
ax1_twin.set_ylabel(Y3_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')

ax1_twin.set_ylim(ymin1_tw, ymax1_tw)

ax1_twin.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))
for k in ['left', 'top', 'bottom']:
    ax1_twin.spines[k].set_visible(False)
ax1_twin.spines['right'].set_visible(True)
ax1_twin.spines['right'].set_color(color_ax1_twin + '6')
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)
ax1_twin.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))

# --- 우측 상단(legend 위치)에 각 series별 텍스트 표시 ---
# 컴프레셔
ax1_twin.text(0.95, 0.3, "Compressor", color=color_ax1_twin + '6',
             fontsize=fs['legend'],
             ha='right', va='bottom',
             transform=ax1_twin.transAxes)
# 펌프
ax1_twin.text(0.95, 0.08, "Pump", color=color_ax1_twin2 + '6',
             fontsize=fs['legend'],
             ha='right', va='bottom',
             transform=ax1_twin.transAxes)

ax1.set_zorder(1)             # 메인 축을 위로
ax1_twin.set_zorder(3)        # 트윈 축을 아래로
ax1.patch.set_alpha(0.0)      # 메인 축 배경을 투명하게 (아니면 ax1.set_facecolor('none'))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig(f'figure/HeatPump_model/E_cmp_&_E_pmp_{xmin1}.svg', dpi=900, transparent=True)
dm.save_and_show(fig)

In [12]:
COP_no_nan = np.nan_to_num(COP, nan=0)
COP_positive = COP_no_nan[COP_no_nan > 0]
min_COP = COP_positive.min() if COP_positive.size > 0 else 0
max_COP = COP_no_nan.max()
print(f"min COP (excluding 0): {min_COP}, max COP: {max_COP}")


print(f"Tank water min max: {T_tank_w.min()} {T_tank_w.max()}")

min COP (excluding 0): 2.1792934566266973, max COP: 3.787177192534502
Tank water min max: 36.714044454497866 65.81584436958059


# 6. refrigerant snapshot video

In [13]:
import os
import cv2
from natsort import natsorted

In [465]:
path = './video'
out_path = '.'
out_video_name = 'refrigerant_cycle_operation'
out_video_full_path = f"{out_path}/{out_video_name}.mp4"


In [44]:
pre_imgs = os.listdir(path)

# 2) 자연 정렬 (문자열이라 key 불필요)
pre_imgs = natsorted(pre_imgs)
pre_imgs = pre_imgs[:200]  # .DS_Store 등 불필요한 파일 제거
# 3) 전체 경로로 변환
img = [os.path.join(path, f) for f in pre_imgs]
print(img)

['./video/0000.png', './video/0001.png', './video/0002.png', './video/0003.png', './video/0004.png', './video/0005.png', './video/0006.png', './video/0007.png', './video/0008.png', './video/0009.png', './video/0010.png', './video/0011.png', './video/0012.png', './video/0013.png', './video/0014.png', './video/0015.png', './video/0016.png', './video/0017.png', './video/0018.png', './video/0019.png', './video/0020.png', './video/0021.png', './video/0022.png', './video/0023.png', './video/0024.png', './video/0025.png', './video/0026.png', './video/0027.png', './video/0028.png', './video/0029.png', './video/0030.png', './video/0031.png', './video/0032.png', './video/0033.png', './video/0034.png', './video/0035.png', './video/0036.png', './video/0037.png', './video/0038.png', './video/0039.png', './video/0040.png', './video/0041.png', './video/0042.png', './video/0043.png', './video/0044.png', './video/0045.png', './video/0046.png', './video/0047.png', './video/0048.png', './video/0049.png',

In [45]:
# 코덱 설정 및 size 설정 
cv2_fourcc = cv2.VideoWriter_fourcc(*'mp4v') # * -> 리스트 언패킹(unpacking)을 위한 문법적인 요소
frame = cv2.imread(img[0]) # frame (높이, 너비, 채널)
size = list(frame.shape) # [높이, 너비, 채널]
del size[2] # [높이, 너비]
size.reverse() # [너비, 높이] 

In [47]:
# VideoWriter(output video name, fourcc, fps, size)
video = cv2.VideoWriter(out_video_full_path, cv2_fourcc, 20, size)

for i in tqdm(range(len(img))):
    video.write(cv2.imread(img[i])) # 비디오 프레임 작성
    # print('frame', i+1, 'of', len(img)) # 몇 번 프레임 작업중인지 알려주는 함수
video.release()

100%|██████████| 200/200 [00:05<00:00, 38.25it/s]


# Test

## Sankey diagram

In [520]:
def plot_sankey(idx):
    # --- 1. Node 정의 (file_context_0와 대응) ---
    labels = [
        # 메인 플로우 (0-6, 실제 변수명)
        "Ground",    # 0
        "Borehole",        # 1
        "Borehole fluid",  # 2
        "Refrigerant\nstate 1",                        # 3
        "Refrigerant\nstate 2",                        # 4
        "Refrigerant\nstate 3",                        # 5
        "Refrigerant\nstate 4",                        # 6
        # 주요 분기/노드
        "Condensor",   # 7
        "Tank water",     # 8
        "Service hot water", # 9
        # 외부 입력
        "Pump power",     # 10
        "Compressor power",   # 11
        "Tank supply water",        # 12
        "Mixing supply water",         # 13
        # 손실/소모 (file_context_0와 변수명 싱크)
        "Ground Xc",           # 14
        "Borehole Xc",           # 15
        "GHE Xc",           # 16
        "Evaporator Xc",              # 17
        "Compressor Xc",               # 18
        "Condensor Xc",          # 19
        "Expansion valve Xc",             # 20
        "Tank Xc",              # 21
        "Mixing valve Xc"          # 22
    ]

    # --- 2. Source-Target 매핑 및 값 할당 (file_context_0 기준, 변수명이 배치되는 위치와 의미가 일치하도록) ---
    # 순서: [source_index] → [target_index], value 인덱스에 대응
    source = [
        # 왼쪽(지중/보어홀)
        0,    # X_g → X_b
        0,    # X_g → Xc_g
        1,    # X_b → X_b_f
        1,    # X_b → Xc_b
        10,   # X_pmp → X_b_f
        2,    # X_b_f → X4
        2,    # X_b_f → Xc_wloop
        # 메인 루프
        5,    # X3 → X4 (루프)
        6,    # X4 → X1
        6,    # X4 → Xc_evap
        11,   # X_cmp → X1
        3,    # X1 → X2
        3,    # X1 → Xc_cmp
        4,    # X2 → X3
        4,    # X2 → X_ref_cond
        4,    # X2 → Xc_ref_cond
        5,    # X3 → Xc_exp
        # 오른쪽(저탕조~믹싱)
        12,   # X_tank_sup_w → X_ref_cond
        7,    # X_ref_cond → X_tank_w
        7,    # X_ref_cond → Xc_tank
        13,   # X_mix_sup_w → X_tank_w
        8,    # X_tank_w → X_mix_serv_w
        8     # X_tank_w → Xc_mix
    ]

    target = [
        1,    # X_g → X_b
        14,   # X_g → Xc_g
        2,    # X_b → X_b_f
        15,   # X_b → Xc_b
        2,    # X_pmp → X_b_f
        6,    # X_b_f → X4
        16,   # X_b_f → Xc_wloop
        # 메인 루프
        6,    # X3 → X4
        3,    # X4 → X1
        17,   # X4 → Xc_evap
        3,    # X_cmp → X1
        4,    # X1 → X2
        18,   # X1 → Xc_cmp
        5,    # X2 → X3
        7,    # X2 → X_ref_cond
        19,   # X2 → Xc_ref_cond
        20,   # X3 → Xc_exp
        # 오른쪽(저탕조~믹싱)
        7,    # X_tank_sup_w → X_ref_cond
        8,    # X_ref_cond → X_tank_w
        21,   # X_ref_cond → Xc_tank
        8,    # X_mix_sup_w → X_tank_w
        9,    # X_tank_w → X_mix_serv_w
        22    # X_tank_w → Xc_mix
    ]

    # --- 3. 값 할당 (file_context_0에서 NumPy 배열/스칼라 변수로 구현된 순서와 일치, [0] 인덱스로 예시 값 추출) ---
    # 실제 사용시 아래 값들은 time step별로 각 변수(X_b, Xc_g 등)의 적절한 NumPy 배열 혹은 스칼라를 [0] 등 인덱스로 전달해야 함
    values = [
        float(X_b[idx]),              # X_g → X_b
        float(Xc_g[idx]),             # X_g → Xc_g
        float(X_b_f[idx]),            # X_b → X_b_f
        float(Xc_b[idx]),             # X_b → Xc_b
        float(X_pmp[idx]),            # X_pmp → X_b_f
        float(X_ref_evap[idx]),     # X_b_f → X4
        float(Xc_GHE[idx]),         # X_b_f → Xc_wloop
        # 메인 루프
        float(X_flow_X3_to_X4[idx]),  # X3 → X4
        float(X_flow_X4_to_X1[idx]),  # X4 → X1
        float(Xc_evap[idx]),          # X4 → Xc_evap
        float(X_cmp[idx]),            # X_cmp → X1
        float(X_flow_X1_to_X2[idx]),  # X1 → X2
        float(Xc_cmp[idx]),           # X1 → Xc_cmp
        float(X_flow_X2_to_X3[idx]),  # X2 → X3
        float(X_ref_cond[idx]),       # X2 → X_ref_cond
        float(Xc_ref_cond[idx]),      # X2 → Xc_ref_cond
        float(Xc_exp[idx]),           # X3 → Xc_exp
        # 오른쪽(저탕조~믹싱)
        float(X_tank_sup_w[idx]),     # X_tank_sup_w → X_ref_cond
        float(X_tank_w[idx]),       # X_ref_cond → X_tank_w
        float(Xc_tank[idx]),          # X_ref_cond → Xc_tank
        float(X_mix_sup_w[idx]),      # X_mix_sup_w → X_tank_w
        float(X_mix_serv_w[idx]),     # X_tank_w → X_mix_serv_w
        float(Xc_mix[idx])            # X_tank_w → Xc_mix
    ]

    # --- 4. 도식화 ---
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels, 
        ),
        link=dict(
            source=source,
            target=target,
            value=values
        )
    )])

    fig.update_layout(
        font_size=12,
        width=1200,
        height=700,
        legend_orientation="h",
        legend_yanchor="bottom",
        legend_y=-0.2,
        legend_xanchor="center",
        legend_x=0.5
    )

    fig.show()

## 6.2 Exergy consumption

In [14]:
def plot_exergy_consumption_barh(idx):
    """
    특정 타임스텝(idx)에 대한 엑서지 소비율 항들을 가로방향 바막대기 플롯(역순)으로 시각화

    Args:
        idx (int): 타임스텝 인덱스
    """
    # 엑서지 소비율 항들과 서브시스템 라벨
    exergy_consumption = [
        Xc_g[idx],
        Xc_b[idx],
        Xc_GHE[idx],
        Xc_evap[idx],
        Xc_cmp[idx],
        Xc_ref_cond[idx],
        Xc_exp[idx],
        Xc_tank[idx]-X_tank_l[idx],
        X_tank_l[idx],
        Xc_mix[idx]
    ]

    subsystem_labels = [
        'Ground',
        'Borehole',
        'GHE loop',
        'Evaporator',
        'Compressor',
        'Condensor',
        'Expansion valve',
        'Tank (heat exchange)',
        'Tank (heat loss)',
        'Mixing valve'
    ]

    # 역순으로 정렬 (가장 마지막 것이 바의 맨 위에 오도록)
    exergy_consumption = exergy_consumption[::-1]
    subsystem_labels = subsystem_labels[::-1]

    # 절댓값으로 변환 (로그 스케일을 위해)
    exergy_consumption_abs = np.abs(exergy_consumption)

    # NaN 혹은 음수만 있을 경우 xlim 문제 방지
    finite_mask = np.isfinite(exergy_consumption_abs) & (exergy_consumption_abs > 0)
    if not np.any(finite_mask):
        print("플롯에 사용할 유효한 엑서지 소비율 데이터가 없습니다.")
        return

    max_value = np.nanmax(exergy_consumption_abs[finite_mask])

    # 10의 제곱단위로 최댓값을 넘지 않는 범위로 xlim 자동 설정
    import math
    log10_max = math.ceil(np.log10(max_value)) if max_value > 0 else 0
    xlim_max = 10 ** log10_max
    xlim_min = 10 ** (log10_max - 3) if log10_max >= 3 else 0.1

    # cmap에서 그라디언트 색상 추출
    import matplotlib.cm as cm
    cmap = cm.get_cmap('dm.Blues9')  # 또는 'plasma', 'inferno', 'magma' 등
    n_subsystems = len(subsystem_labels)
    cm_min_ratio, cm_max_ratio = 0.3, 1  # 맨 끝(진한색) 또는 시작(연한색) 일부 구간 제외
    # 색상도 역순(밝은색~진한색 매칭, bar와 label 순서 동일하게)
    colors = [
        cmap(cm_min_ratio + (cm_max_ratio - cm_min_ratio) * (i / (n_subsystems - 1)))
        for i in range(n_subsystems)
    ]

    # 플롯 생성
    fig, ax = plt.subplots(figsize=(dm.cm2in(12), dm.cm2in(5.5)))

    # 바 두께 정보 알아오기
    bar_height = 0.8  # matplotlib 기본 barh 두께 (height), 바꿀 수도 있음

    # margin을 bar 두께의 0.2배로 위아래에 추가. 즉 margin=bar_height*0.2
    margin = bar_height * 0.2

    # ypos 조정 (0~N-1 범위에 margin만큼 위아래로 여유)
    y_pos = np.arange(len(subsystem_labels))
    # 컬럼 수가 많지 않으니, ylim을 조정해서 margin 반영
    ax.set_ylim(-margin - bar_height/2, len(subsystem_labels)-1 + margin + bar_height/2)

    # 가로 바막대기 플롯 (역순 데이터)
    bars = ax.barh(
        y_pos,
        exergy_consumption_abs,
        color=colors,
        linewidth=LW[0],
        height=bar_height
    )

    # 로그 스케일 적용 및 xlim 자동 설정
    ax.set_xscale('log')
    ax.set_xlim(left=xlim_min, right=xlim_max)

    # Y축 설정 (서브시스템 라벨, 역순)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(subsystem_labels, fontsize=fs['tick'])

    # X축 설정
    ax.set_xlabel('Exergy consumption rate [W]', fontsize=fs['label'], labelpad=pad['label'])

    # 축 스타일 설정
    ax.tick_params(axis = 'y', labelsize=fs['tick'], which='major', length=0, width=0.3, pad=pad['tick'])
    ax.tick_params(axis = 'x', labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
    ax.tick_params(axis = 'x', labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])
    # x축 마이너틱을 10개로 설정
    import matplotlib.ticker as mticker
    ax.xaxis.set_minor_locator(mticker.LogLocator(subs=np.linspace(1, 10, 10), numticks=10))

    # 바 끝에 숫자 값(정수, 소수점 없이) 표기
    for bar, value in zip(bars, exergy_consumption_abs):
        width = bar.get_width()
        ypos = bar.get_y() + bar.get_height() / 2
        if not np.isnan(width) and width > 0:
            ax.text(
                width * 1.03,
                ypos,
                f"{int(round(width))}" if width > 10 else f"{width:.1f}",
                va='center',
                ha='left',
                fontsize=fs['text'],
                color='k'
            )

    # 스파인 설정
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    for spine in ['left', 'bottom']:
        ax.spines[spine].set_visible(True)

    # 그리드 추가 (선택사항)
    ax.grid(True, axis='x', alpha=0.3, linestyle='--', linewidth=LW[0])

    # 레이아웃 조정
    hour_idx = idx * dt * dem.s2h
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=(0.01, 1, 0, 1), verbose=False)
    plt.savefig(f'figure/exergy_consumption_barh_{hour_idx}.svg', dpi=900, transparent=True)
    plt.savefig(f'figure/exergy_consumption_barh_{hour_idx}.png', dpi=900, transparent=True)
    dm.save_and_show(fig)

In [17]:
def plot_exergy_consumption_barh_ratio(idx):
    """
    특정 타임스텝(idx)에 대한 엑서지 소비율 항들을 가로방향 바막대기 플롯(역순)으로 시각화

    Args:
        idx (int): 타임스텝 인덱스
    """
    # 엑서지 소비율 항들과 서브시스템 라벨
    exergy_consumption = [
        Xc_g[idx]/X_g[idx],
        Xc_b[idx]/X_b[idx],
        Xc_GHE[idx]/(X_pmp[idx] + X_b_f[idx]),
        Xc_evap[idx]/(X4[idx] + X_ref_evap[idx]),
        Xc_cmp[idx]/(X1[idx] + X_cmp[idx]),
        Xc_ref_cond[idx]/(X_ref_cond[idx] + X2[idx]),
        Xc_exp[idx]/(X3[idx]),
        (Xc_tank[idx]-X_tank_l[idx])/(X_tank_sup_w[idx] + X_ref_cond[idx]),
        X_tank_l[idx]/(X_tank_sup_w[idx] + X_ref_cond[idx]),
        Xc_mix[idx]/(X_mix_sup_w[idx] + X_mix_serv_w[idx])
    ]

    subsystem_labels = [
        'Ground',
        'Borehole',
        'GHE loop',
        'Evaporator',
        'Compressor',
        'Condensor',
        'Expansion valve',
        'Tank (heat exchange)',
        'Tank (heat loss)',
        'Mixing valve'
    ]

    # 역순으로 정렬 (가장 마지막 것이 바의 맨 위에 오도록)
    exergy_consumption = exergy_consumption[::-1]
    subsystem_labels = subsystem_labels[::-1]

    # 절댓값으로 변환 (로그 스케일을 위해)
    exergy_consumption_abs = np.abs(exergy_consumption)*100 # 백분율로 변환

    # NaN 혹은 음수만 있을 경우 xlim 문제 방지
    finite_mask = np.isfinite(exergy_consumption_abs) & (exergy_consumption_abs > 0)
    if not np.any(finite_mask):
        print("플롯에 사용할 유효한 엑서지 소비율 데이터가 없습니다.")
        return

    max_value = np.nanmax(exergy_consumption_abs[finite_mask])

    # 10의 제곱단위로 최댓값을 넘지 않는 범위로 xlim 자동 설정
    import math
    log10_max = math.ceil(np.log10(max_value)) if max_value > 0 else 0
    xlim_max = 60
    xlim_min = 0

    # cmap에서 그라디언트 색상 추출
    import matplotlib.cm as cm
    cmap = cm.get_cmap('dm.Blues9')  # 또는 'plasma', 'inferno', 'magma' 등
    n_subsystems = len(subsystem_labels)
    cm_min_ratio, cm_max_ratio = 0.3, 1  # 맨 끝(진한색) 또는 시작(연한색) 일부 구간 제외
    # 색상도 역순(밝은색~진한색 매칭, bar와 label 순서 동일하게)
    colors = [
        cmap(cm_min_ratio + (cm_max_ratio - cm_min_ratio) * (i / (n_subsystems - 1)))
        for i in range(n_subsystems)
    ]

    # 플롯 생성
    fig, ax = plt.subplots(figsize=(dm.cm2in(12), dm.cm2in(5.5)))

    # 바 두께 정보 알아오기
    bar_height = 0.8  # matplotlib 기본 barh 두께 (height), 바꿀 수도 있음

    # margin을 bar 두께의 0.2배로 위아래에 추가. 즉 margin=bar_height*0.2
    margin = bar_height * 0.2

    # ypos 조정 (0~N-1 범위에 margin만큼 위아래로 여유)
    y_pos = np.arange(len(subsystem_labels))
    # 컬럼 수가 많지 않으니, ylim을 조정해서 margin 반영
    ax.set_ylim(-margin - bar_height/2, len(subsystem_labels)-1 + margin + bar_height/2)

    # 가로 바막대기 플롯 (역순 데이터)
    bars = ax.barh(
        y_pos,
        exergy_consumption_abs,
        color=colors,
        linewidth=LW[0],
        height=bar_height
    )

    # 로그 스케일 적용 및 xlim 자동 설정
    ax.set_xlim(left=xlim_min, right=xlim_max)

    # Y축 설정 (서브시스템 라벨, 역순)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(subsystem_labels, fontsize=fs['tick'])

    # X축 설정
    ax.set_xlabel('Exergy consumption rate [W]', fontsize=fs['label'], labelpad=pad['label'])

    # 축 스타일 설정
    ax.tick_params(axis = 'y', labelsize=fs['tick'], which='major', length=0, width=0.3, pad=pad['tick'])
    ax.tick_params(axis = 'x', labelsize=fs['tick'], which='major', length=2.5, width=0.3, pad=pad['tick'])
    ax.tick_params(axis = 'x', labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])
    # x축 마이너틱을 10개로 설정

    # 바 끝에 숫자 값(정수, 소수점 없이) 표기
    for bar, value in zip(bars, exergy_consumption_abs):
        width = bar.get_width()
        ypos = bar.get_y() + bar.get_height() / 2
        if not np.isnan(width) and width > 0:
            ax.text(
                width * 1.03,
                ypos,
                f"{int(round(width))}" if width > 10 else f"{width:.1f}",
                va='center',
                ha='left',
                fontsize=fs['text'],
                color='k'
            )

    # 스파인 설정
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    for spine in ['left', 'bottom']:
        ax.spines[spine].set_visible(True)

    # 그리드 추가 (선택사항)
    ax.grid(True, axis='x', alpha=0.3, linestyle='--', linewidth=LW[0])

    # 레이아웃 조정
    hour_idx = idx * dt * dem.s2h
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=(0.01, 1, 0, 1), verbose=False)
    plt.savefig(f'figure/exergy_consumption_barh_{hour_idx}.svg', dpi=900, transparent=True)
    plt.savefig(f'figure/exergy_consumption_barh_{hour_idx}.png', dpi=900, transparent=True)
    dm.save_and_show(fig)

In [18]:
# 원하는 시간(단위: hour)을 입력하세요. 예시: 원하는 시간 = 80시간
target_hour = 960.5 + 8

# 플롯에 사용할 데이터 마스크: 공급 유량 변화 O, 시스템 ON
mask = (dV_w_sup_tank != 0) & (is_on == True)
nonzero_indices = np.where(mask)[0]
hours = time * dem.s2h

if len(nonzero_indices) == 0:
    print("조건을 만족하는 인덱스가 없습니다.")
else:
    # nonzero_indices가 가리키는 hours 배열 값들 중 target_hour와 가장 가까운 인덱스 찾기
    target_idx = nonzero_indices[np.argmin(np.abs(hours[nonzero_indices] - target_hour))]
    print(f"입력시간={target_hour} h에 가장 근접한 index={target_idx}, 시간={hours[target_idx]:.2f} h")
    print(f"저탕조 온도: {T_tank_w[target_idx]:.1f} °C")
    print(f"사용 온수 유량: {dV_w_serv[target_idx]*60000:.1f} L/min")
    print(f"사용 온수 엑서지율: {X_mix_serv_w[target_idx]:.1f} W")
    plot_exergy_consumption_barh(target_idx)
if len(nonzero_indices) == 0:
    print("조건을 만족하는 인덱스가 없습니다.")
else:
    # nonzero_indices가 가리키는 hours 배열 값들 중 target_hour와 가장 가까운 인덱스 찾기
    target_idx = nonzero_indices[np.argmin(np.abs(hours[nonzero_indices] - target_hour))]
    print(f"입력시간={target_hour} h에 가장 근접한 index={target_idx}, 시간={hours[target_idx]:.2f} h")
    print(f"저탕조 온도: {T_tank_w[target_idx]:.1f} °C")
    print(f"사용 온수 유량: {dV_w_serv[target_idx]*60000:.1f} L/min")
    print(f"사용 온수 엑서지율: {X_mix_serv_w[target_idx]:.1f} W")
    plot_exergy_consumption_barh_ratio(target_idx)

입력시간=968.5 h에 가장 근접한 index=58110, 시간=968.50 h
저탕조 온도: 44.6 °C
사용 온수 유량: 1.8 L/min
사용 온수 엑서지율: 335.4 W


/tmp/ipykernel_1170245/3313721673.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('dm.Blues9')  # 또는 'plasma', 'inferno', 'magma' 등


입력시간=968.5 h에 가장 근접한 index=58110, 시간=968.50 h
저탕조 온도: 44.6 °C
사용 온수 유량: 1.8 L/min
사용 온수 엑서지율: 335.4 W


/tmp/ipykernel_1170245/3898029835.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('dm.Blues9')  # 또는 'plasma', 'inferno', 'magma' 등


In [19]:
plot_exergy_consumption_barh_ratio(target_idx)

/tmp/ipykernel_1170245/3898029835.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('dm.Blues9')  # 또는 'plasma', 'inferno', 'magma' 등


In [421]:
# dV_w_serv가 정의되어 있다고 가정 (예: NumPy 배열)
mask = (dV_w_sup_tank != 0) & (is_on == True)

# 해당 인덱스, 시간(단위: 시간) 프린트
nonzero_indices = np.where(mask)[0]
hours = time * dem.s2h
idx = nonzero_indices[2]
print(f"index={idx}, 시간={hours[idx]:.2f} h")

print(f"X_tank_w[idx]: {X_tank_w[idx]:.2f}")
print(f"X_tank_sup_w[idx]: {X_tank_sup_w[idx]:.2f}")
print(f"Xc_tank[idx]: {Xc_tank[idx]:.2f}")

plot_sankey(idx)

index=375, 시간=6.25 h
X_tank_w[idx]: 642.66
X_tank_sup_w[idx]: 51.36
Xc_tank[idx]: 759.42


In [ ]:
# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'result/gshpb_simulation_results2.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = time * dem.s2h  # 시간 [h]
Y1 = T_b
Y2 = Xc_g

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Borehole wall temp [°C]'
Y2_LABEL = 'Ground exergy consum [W]'

xmin1 = 960
xmax1, xint1, xmar1 = xmin1+24, 12, 0
ymin1, ymax1, yint1, ymar1 = 12, 17, 1, 0
ymin1_tw, ymax1_tw, yint1_tw = 0, 70, 10
color_ax1 = 'dm.red'
color_ax1_twin = 'dm.orange'

# ===============================================================
mask_1 = (X >= 0) & (X <= 24)
mask_2 = (X >= 960) & (X <= 984)

fig, (axL, axR) = plt.subplots(
    1, 2, figsize=(dm.cm2in(16), dm.cm2in(5)),
    gridspec_kw={'wspace': 0.2}
)

# Left panel (72–96 h): show ONLY left spine/ticks
axL.plot(X[mask_1], Y1[mask_1], color=color_ax1 + '4', linewidth=LW[2], rasterized=True,zorder=4)
axL.set_xlim(0, 24)
axL.set_ylim(ymin1, ymax1)
axL.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
axL.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')
axL.set_xticks(np.arange(0, 24.001, 6))
axL.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
axL.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))
axL.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='both', pad=pad['tick'])
axL.tick_params(axis='x', labelsize=fs['tick'], which='both', pad=pad['tick'])
# Hide right spine on left panel
axL.spines['right'].set_visible(False)
axL.spines['left'].set_color(color_ax1 + '6')

axL_r = axL.twinx()
axL_r.fill_between(X[mask_1], 0, Y2[mask_1], color=color_ax1_twin + '2', alpha=0.5, rasterized=True, zorder=0)
axL_r.set_ylim(ymin1_tw, ymax1_tw)
# Completely hide right axis visuals
axL_r.spines['left'].set_visible(False)
axL_r.spines['right'].set_visible(False)
axL_r.set_ylabel('')
axL_r.set_yticks([])

axL.set_zorder(2)
axL_r.set_zorder(1)
axL.patch.set_alpha(0.0)

# Right panel (960–984 h): show ONLY right spine/ticks
axR.plot(X[mask_2], Y1[mask_2], color=color_ax1 + '4', linewidth=LW[2], linestyle='-', rasterized=True,zorder=4)
axR.set_xlim(xmin1, xmax1)
axR.set_ylim(ymin1, ymax1)
axR.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
# Hide left spine/ticks on right panel
axR.spines['left'].set_visible(False)
axR.set_ylabel('')
axR.set_yticks([])
axR.set_xticks(np.arange(960, 984.001, 6))
axR.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
axR.tick_params(axis='x', labelsize=fs['tick'], which='both', pad=pad['tick'])

axR_r = axR.twinx()
axR_r.fill_between(X[mask_2], 0, Y2[mask_2], color=color_ax1_twin + '2', alpha=0.5, rasterized=True, zorder=0)
axR_r.set_ylabel(Y2_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')
axR_r.set_ylim(ymin1_tw, ymax1_tw)
axR_r.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))
axR_r.yaxis.set_minor_locator(ticker.AutoMinorLocator(1))
axR_r.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='both', pad=pad['tick'])
axR_r.spines['left'].set_visible(False)
axR_r.spines['right'].set_visible(True)
axR_r.spines['right'].set_color(color_ax1_twin + '6')

axR.set_zorder(2)
axR_r.set_zorder(1)
axR.patch.set_alpha(0.0)

dm.simple_layout(fig, bbox=[0, 1, 0.02, 1], margins=[0.0, 0.0, 0.0, 0.0])
plt.savefig('figure/HeatPump_model/Borehole_wall_temp_&_ground_Xc_split.svg', dpi=900, transparent=True)
plt.savefig('figure/HeatPump_model/Borehole_wall_temp_&_ground_Xc_split.png', dpi=900, transparent=True)
dm.save_and_show(fig)
# ...existing code...